In [1]:
# Reset Cell - for if kernel closed/restarted, resets all needed variables
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
from ipaddress import ip_address, IPv4Address, IPv4Network
fablib = fablib_manager()
slice_name = "bmv2-ra-demo"
slice = fablib.get_slice(name=slice_name)
h1 = slice.get_node(name="h1")
h2 = slice.get_node(name="h2")
s1 = slice.get_node(name="s1")
hosts = [h1, h2]
h1_iface0_name = h1.get_interface(network_name="h1_s1").get_device_name()
h2_iface0_name = h2.get_interface(network_name="h2_s1").get_device_name()
s1_iface0_name = s1.get_interface(network_name="h1_s1").get_device_name()
s1_iface1_name = s1.get_interface(network_name="h2_s1").get_device_name()
h1_ip = IPv4Address("192.187.1.11")
h2_ip = IPv4Address("192.187.1.12")
s1_ip = IPv4Address("192.187.1.1")
h1_macs = ["00:00:00:00:00:11"]
h2_macs = ["00:00:00:00:00:12"]
s1_macs = ["00:00:00:00:00:01", "00:00:00:00:00:02"]
subnet = IPv4Network("192.187.16.0/24")
ips = {
    "h1": h1_ip,
    "h2": h2_ip,
    "s1": s1_ip
}
macs = {
    "h1": h1_macs,
    "h2": h2_macs,
    "s1": s1_macs
}
ifaces = {
    "h1": [h1_iface0_name],
    "h2": [h2_iface0_name],
    "s1": [s1_iface0_name, s1_iface1_name]
}

In [9]:
#slice.delete()

In [10]:
# Build Slice
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
slice_name = "bmv2-ra-demo"
slice = fablib.new_slice(name=slice_name)
host_cores = 2
host_ram = 2
host_disk = 20
switch_cores = 4
switch_ram = 8
switch_disk = 50
image = "default_ubuntu_20"
site = "KANS"
h1 = slice.add_node(name="h1", site=site, cores=host_cores, ram=host_ram, disk=host_disk, image=image)
h1_iface0 = h1.add_component(model="NIC_Basic", name="h1p0").get_interfaces()[0]
h2 = slice.add_node(name="h2", site=site, cores=host_cores, ram=host_ram, disk=host_disk, image=image)
h2_iface0 = h2.add_component(model="NIC_Basic", name="h2p0").get_interfaces()[0]
s1 = slice.add_node(name="s1", site=site, cores=switch_cores, ram=switch_ram, disk=switch_disk, image=image)
s1_iface0 = s1.add_component(model="NIC_Basic", name="s1p0").get_interfaces()[0]
s1_iface1 = s1.add_component(model="NIC_Basic", name="s1p1").get_interfaces()[0]
h1_s1 = slice.add_l2network(name="h1_s1", interfaces=[h1_iface0, s1_iface0])
h2_s1 = slice.add_l2network(name="h2_s1", interfaces=[h2_iface0, s1_iface1])
slice.submit()
hosts = [h1, h2]
h1 = slice.get_node(name="h1")
h2 = slice.get_node(name="h2")
s1 = slice.get_node(name="s1")
h1_iface0_name = h1.get_interface(network_name="h1_s1").get_device_name()
h2_iface0_name = h2.get_interface(network_name="h2_s1").get_device_name()
s1_iface0_name = s1.get_interface(network_name="h1_s1").get_device_name()
s1_iface1_name = s1.get_interface(network_name="h2_s1").get_device_name()


Retry: 10, Time: 267 sec


ID,910ed1ba-d54b-4ca7-babc-163ee8a73a1a
Name,bmv2-ra-demo
Lease Expiration (UTC),2024-06-06 19:22:21 +0000
Lease Start (UTC),2024-06-05 19:22:22 +0000
Project ID,f8791874-1b0a-49bf-bc4d-7142be017468
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
851569c8-e428-433f-8627-1c3edb5ae0ca,h1,2,2,100,default_ubuntu_20,qcow2,kans-w3.fabric-testbed.net,KANS,ubuntu,2001:400:a100:3060:f816:3eff:fee1:7056,Active,,ssh -i /home/fabric/work/fabric_config/fpga-silver -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3060:f816:3eff:fee1:7056,/home/fabric/work/fabric_config/fpga-silver.pub,/home/fabric/work/fabric_config/fpga-silver
ace8dbf7-9c8e-412f-a8fb-a8575196a6cf,h2,2,2,100,default_ubuntu_20,qcow2,kans-w3.fabric-testbed.net,KANS,ubuntu,2001:400:a100:3060:f816:3eff:fe64:9f7b,Active,,ssh -i /home/fabric/work/fabric_config/fpga-silver -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3060:f816:3eff:fe64:9f7b,/home/fabric/work/fabric_config/fpga-silver.pub,/home/fabric/work/fabric_config/fpga-silver
975da64f-5f85-46d4-9990-2db8fbb8f8c9,s1,4,8,100,default_ubuntu_20,qcow2,kans-w3.fabric-testbed.net,KANS,ubuntu,2001:400:a100:3060:f816:3eff:fe59:d685,Active,,ssh -i /home/fabric/work/fabric_config/fpga-silver -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3060:f816:3eff:fe59:d685,/home/fabric/work/fabric_config/fpga-silver.pub,/home/fabric/work/fabric_config/fpga-silver


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
d1fc1200-6e86-4786-98c0-9fbf4cee0066,h1_s1,L2,L2Bridge,KANS,None,None,Active,
aa755ca7-6daa-4cf6-ba35-64af47803f44,h2_s1,L2,L2Bridge,KANS,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
h1-h1p0-p1,p1,h1,h1_s1,100,config,,0A:67:48:D4:F2:A5,enp7s0,enp7s0,fe80::867:48ff:fed4:f2a5,4
h2-h2p0-p1,p1,h2,h2_s1,100,config,,0A:7F:3A:7C:C5:48,enp7s0,enp7s0,fe80::87f:3aff:fe7c:c548,4
s1-s1p1-p1,p1,s1,h2_s1,100,config,,0E:E1:FE:DE:8C:D0,enp8s0,enp8s0,fe80::ce1:feff:fede:8cd0,4
s1-s1p0-p1,p1,s1,h1_s1,100,config,,0A:A0:01:7D:D6:EF,enp7s0,enp7s0,fe80::8a0:1ff:fe7d:d6ef,4



Time to print interfaces 281 seconds


In [18]:
# Install net-tools and scapy on all nodes
try:
    install_net_tools = 'sudo apt-get install -y net-tools'
    install_scapy = 'sudo apt-get update && sudo apt-get install -y python3-scapy'
    for node in slice.get_nodes():
        stdout, stderr = node.execute(install_net_tools, quiet=True)
        stdout, stderr = node.execute(install_scapy, quiet=True)
except Exception as e:
    print(f"Exception: {e}")

In [17]:
# Declare addresses and turn up interfaces
from ipaddress import ip_address, IPv4Address, IPv4Network
h1_ip = IPv4Address("192.187.1.11")
h2_ip = IPv4Address("192.187.2.11")
s1p0_ip = IPv4Address("192.187.1.1")
s1p1_ip = IPv4Address("192.187.2.1")
h1_macs = ["00:00:00:00:00:11"]
h2_macs = ["00:00:00:00:00:21"]
s1_macs = ["00:00:00:00:00:01", "00:00:00:00:00:02"]
subnet1 = IPv4Network("192.187.1.0/24")
subnet2 = IPv4Network("192.187.2.0/24")
ips = {
    "h1": [h1_ip],
    "h2": [h2_ip],
    "s1": [s1p0_ip, s1p1_ip]
}
macs = {
    "h1": h1_macs,
    "h2": h2_macs,
    "s1": s1_macs
}
h1.execute(f"sudo ip link set dev {h1_iface0_name} up", quiet=True)
h2.execute(f"sudo ip link set dev {h2_iface0_name} up", quiet=True)
s1.execute(f"sudo ip link set dev {s1_iface0_name} up", quiet=True)
s1.execute(f"sudo ip link set dev {s1_iface1_name} up", quiet=True)
s1.execute("sudo sysctl net.ipv4.ip_forward=1", quiet=True)
ifaces = {
    "h1": [h1_iface0_name],
    "h2": [h2_iface0_name],
    "s1": [s1_iface0_name, s1_iface1_name]
}

In [19]:
# Set iface addresses, ARP, and routes
for node in slice.get_nodes():
    nodename = node.get_name()
    c = len(ifaces[nodename])
    for i in range(0, c):
        node.execute(f"sudo ifconfig {ifaces[nodename][i]} hw ether {macs[nodename][i]}")
        node.execute(f"sudo ifconfig {ifaces[nodename][i]} {ips[nodename][i]}")
        # Disable ARP to reduce traffic pollution
        if nodename == "s1":
            node.execute(f"sudo ip link set dev {ifaces[nodename][i]} arp off")
    if nodename == "h1":
        #node.execute(f"sudo ip route add {s1p0_ip} dev {ifaces[nodename][0]}")
        node.execute(f"sudo ip route add {subnet2} via {ips['s1'][0]}")
        node.execute(f"sudo arp -s {ips['s1'][0]} {macs['s1'][0]}")
    if nodename == "h2":
        #node.execute(f"sudo ip route add {s1_ip} dev {ifaces[nodename][0]}")
        node.execute(f"sudo ip route add {subnet1} via {ips['s1'][1]}")
        node.execute(f"sudo arp -s {ips['s1'][1]} {macs['s1'][1]}")
    if nodename == "s1":
        node.execute(f"sudo ip route del {subnet1}")
        node.execute(f"sudo ip route del {subnet2}")
        

In [20]:
# Build upload file which will put scripts on hosts
def get_scp(nodename):
    ssh = slice.get_node(name=nodename).get_ssh_command()
    param = ssh.split("ssh ")[1]
    scp = "scp " + param
    scp_addr = '[' + scp.split('@')[1] + ']'
    scp_config = scp.split('@')[0]
    scp_base = scp_config + '@' + scp_addr
    return scp_base

def craft_scp(scp_base, file):
    return scp_base.split("ubuntu")[0] + file + " ubuntu" + scp_base.split("ubuntu")[1] + ":~"

s1_bmv2 = "run_bmv2_s1.sh"
with open(s1_bmv2, 'w') as out:
    out.write("#!/bin/bash\n\n")
    out.write(f"sudo ~/bmv2-remote-attestation/targets/simple_switch/simple_switch ")
    out.write(f"-i 0@{s1_iface0_name} -i 1@{s1_iface1_name} ~/router.json --log-console -- --enable-swap")

scps = []
outfile = "uploader.sh"
scp_s1 = get_scp(s1.get_name())
scps.append(craft_scp(scp_s1, "router.p4"))
scps.append(craft_scp(scp_s1, "router2.p4"))
scps.append(craft_scp(scp_s1, "s1_setup.sh"))
scps.append(craft_scp(scp_s1, "run_bmv2_s1.sh"))
scps.append(craft_scp(scp_s1, "add_rules_s1.sh"))
with open(outfile, 'w') as out:
    out.write("#!/bin/bash\n\n")
    for command in scps:
        out.write(command+'\n')

In [21]:
%%sh
chmod u+x uploader.sh
./uploader.sh

In [22]:
# Setup S1
s1.execute("chmod u+x s1_setup.sh run_bmv2_s1.sh add_rules_s1.sh", quiet=True)
s1.execute("./s1_setup.sh", quiet=True)
s1.execute("p4c --target bmv2 --arch v1model router.p4", quiet=True)
s1.execute("p4c --target bmv2 --arch v1model router2.p4", quiet=True)

deb http://download.opensuse.org/repositories/home:/p4lang/xUbuntu_20.04/ /
 Cloning into 'bmv2-remote-attestation'...
 Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://download.opensuse.org/repositories/home:/p4lang/xUbuntu_20.04  InRelease [1542 B]
Hit:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:5 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease
Get:6 http://download.opensuse.org/repositories/home:/p4lang/xUbuntu_20.04  Packages [1929 B]
Fetched 3471 B in 1s (6511 B/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  autoconf automake autotools-dev binfmt-support binutils binutils-common
  binutils-x86-64-linux-gnu clang clang-10 cpp cpp-8 cpp-9 flex gcc gcc-7-base
  gcc-8 gcc-8-base gcc-9 gcc-9-base gfortran gfortran-8 gfortr

('deb http://download.opensuse.org/repositories/home:/p4lang/xUbuntu_20.04/ /\nHit:1 http://security.ubuntu.com/ubuntu focal-security InRelease\nHit:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease\nGet:3 http://download.opensuse.org/repositories/home:/p4lang/xUbuntu_20.04  InRelease [1542 B]\nHit:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease\nHit:5 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease\nGet:6 http://download.opensuse.org/repositories/home:/p4lang/xUbuntu_20.04  Packages [1929 B]\nFetched 3471 B in 1s (6511 B/s)\nReading package lists...\nReading package lists...\nBuilding dependency tree...\nReading state information...\nThe following additional packages will be installed:\n  autoconf automake autotools-dev binfmt-support binutils binutils-common\n  binutils-x86-64-linux-gnu clang clang-10 cpp cpp-8 cpp-9 flex gcc gcc-7-base\n  gcc-8 gcc-8-base gcc-9 gcc-9-base gfortran gfortran-8 gfortran-9\n  ibverbs-providers i

In [3]:
# Get the tpcdump command for h1p0 to see RA packets
print(f"sudo tcpdump -vvvxxe -i {ifaces['h1'][0]}")

sudo tcpdump -vvvxxe -i enp7s0


In [23]:
slice = fablib.get_slice(name=slice_name)
for node in slice.get_nodes():
    print(f"{node.get_name()}: {node.get_ssh_command()}")

h1: ssh -i /home/fabric/work/fabric_config/fpga-silver -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3060:f816:3eff:fee1:7056
h2: ssh -i /home/fabric/work/fabric_config/fpga-silver -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3060:f816:3eff:fe64:9f7b
s1: ssh -i /home/fabric/work/fabric_config/fpga-silver -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3060:f816:3eff:fe59:d685


In [24]:
# Extend the lease as needed
do_extend = False
from datetime import datetime, timedelta
from dateutil import tz
try:
    if do_extend:
        slice = fablib.get_slice(name=slice_name)
        end_date = (datetime.now(tz=tz.tzutc()) + timedelta(days=3)).strftime("%Y-%m-%d %H:%M:%S %z")
        slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")

In [25]:
slice = fablib.get_slice(name=slice_name)
print(slice.get_lease_end())

2024-06-08 19:44:57 +0000
